In [0]:
spark

In [0]:
%sql
-- DROP TABLE poctesting.silver_events;
-- DROP TABLE poctesting.bronze_events;
-- DROP TABLE poctesting.gold_events;

In [0]:
import time

for i in range(150):
    print(f"▶️ Ejecutando iteración {i+1}")
    dbutils.notebook.run("./setup_optimize", timeout_seconds=200)
    time.sleep(1)

In [0]:
# %run ./setup_optimize

In [0]:
# Traemos los datos de prueba bronces
data = spark.table("poctesting.bronze_events")
display(data.limit(5))
print(f"El archivo tiene {data.count()} registros")

In [0]:
from pyspark.sql.functions import col

# Importar bronze
df_bronze = spark.table("poctesting.bronze_events")

# Registros completos
df_completos = df_bronze.filter(
    col("neighborhood").isNotNull() & col("district").isNotNull()
)

# Registros incompletos
df_incompletos = df_bronze.filter(
    col("neighborhood").isNull() | col("district").isNull()
)

In [0]:
from pyspark.sql.functions import col, row_number, count
from pyspark.sql.window import Window
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
from sklearn.neighbors import BallTree
import numpy as np

def corregir_con_sjoin_y_balltree(df_spark, path_parquet_neigh, schema):
    df = df_spark.toPandas()
    if df.empty:
        return spark.createDataFrame([], schema)

    df["geometry"] = df.apply(lambda row: Point(row["longitude"], row["latitude"]), axis=1)
    gdf = gpd.GeoDataFrame(df, geometry="geometry", crs="EPSG:4326")

    gdf_neigh = gpd.read_parquet(path_parquet_neigh)[["NOMBRE", "IDENTIFICACION", "geometry"]]
    gdf_neigh = gdf_neigh.dropna(subset=["geometry"]).set_crs("EPSG:4326")

    gdf = gdf.to_crs("EPSG:3857")
    gdf_neigh = gdf_neigh.to_crs("EPSG:3857")

    gdf_joined = gpd.sjoin(gdf, gdf_neigh, how="left", predicate="within")
    gdf["neighborhood"] = gdf_joined["NOMBRE"]
    gdf["district"] = gdf_joined["IDENTIFICACION"]

    gdf_nulos = gdf[gdf["neighborhood"].isna()].copy()
    if not gdf_nulos.empty:
        centroids = gdf_neigh.geometry.centroid
        neigh_coords = np.array([[pt.y, pt.x] for pt in centroids])
        point_coords = np.array([[pt.y, pt.x] for pt in gdf_nulos.geometry])
        tree = BallTree(np.deg2rad(neigh_coords), metric="haversine")
        dist, idx = tree.query(np.deg2rad(point_coords), k=1)
        gdf.loc[gdf["neighborhood"].isna(), "neighborhood"] = gdf_neigh.iloc[idx.flatten()]["NOMBRE"].values
        gdf.loc[gdf["district"].isna(), "district"] = gdf_neigh.iloc[idx.flatten()]["IDENTIFICACION"].values

    gdf = gdf.dropna(subset=["neighborhood", "district"])

    if gdf.empty:
        return spark.createDataFrame([], schema)
    else:
        return spark.createDataFrame(gdf.drop(columns=["geometry"]))

def actualizar_silver_eventos(df_completos, df_incompletos, path_parquet_neigh):
    nombre_tabla_silver = "poctesting.silver_events"
    schema = df_incompletos.schema

    # Corregir los incompletos
    df_corregido = corregir_con_sjoin_y_balltree(df_incompletos, path_parquet_neigh, schema)
    cantidad_corregidos = df_corregido.count()
    cantidad_completos = df_completos.count()

    # Unir completados
    if df_corregido.limit(1).count() == 0:
        print(f"⚠️ No se corrigieron registros incompletos. Se usará solo df_completos ({cantidad_completos}).")
        df_union = df_completos
    else:
        print(f"✅ Se corrigieron {cantidad_corregidos} registros. Total con completos: {cantidad_completos + cantidad_corregidos}")
        df_union = df_completos.unionByName(df_corregido)

    # Eliminar duplicados por order_id conservando el de mayor quantity_products
    window_spec = Window.partitionBy("order_id").orderBy(col("quantity_products").desc())
    df_union_dedup = df_union.withColumn("rn", row_number().over(window_spec)).filter("rn = 1").drop("rn")

    # Cargar datos existentes (si los hay)
    tabla_existe = spark.catalog.tableExists(nombre_tabla_silver)
    if tabla_existe:
        df_existente = spark.table(nombre_tabla_silver)
        df_todo = df_existente.unionByName(df_union_dedup)

        # Deduplicar final por order_id (mantener mayor quantity_products)
        window_final = Window.partitionBy("order_id").orderBy(col("quantity_products").desc())
        df_final = df_todo.withColumn("rn", row_number().over(window_final)).filter("rn = 1").drop("rn")
    else:
        df_final = df_union_dedup

    # Guardar en tabla silver
    modo = "overwrite" if not tabla_existe else "overwrite"
    df_final.write.mode(modo).saveAsTable(nombre_tabla_silver)

    # Verificación final de duplicados
    df_verif = spark.table(nombre_tabla_silver)
    df_check = df_verif.groupBy("order_id").agg(count("*").alias("cantidad")).filter("cantidad > 1")

    if df_check.count() > 0:
        print(f"❌ Duplicados encontrados en 'order_id': {df_check.count()}")
        display(df_check)
    else:
        print(f"✅ Tabla 'silver_eventos' actualizada correctamente con {df_final.count()} registros únicos por orden.")


In [0]:
path_parquet_neigh = "/Workspace/Users/danielale22rojas@gmail.com//medellin-bigdata-poc/data/raw/medellin_neighborhoods.parquet"
actualizar_silver_eventos(df_completos, df_incompletos, path_parquet_neigh)

In [0]:
df_silver = spark.table("poctesting.silver_events")
# df_silver.write.format("delta").mode("overwrite").saveAsTable("poctesting.silver_events")
print(f"Descartamos {df_bronze.count() - df_silver.count()} registros con información faltante")

In [0]:
%sql
-- cual es el numero promedio de produtos vendidos
-- SELECT avg(quantity_products) FROM poctesting.silver_events;
-- SELECT avg(quantity_products) FROM poctesting.bronze_events;

### Contrucción de ventanas para gold

In [0]:

# Ventanas por barrio y empleado
window_neigh = Window.partitionBy("neighborhood")
window_emp_total = Window.partitionBy("employee_id")
window_rank_global = Window.orderBy(F.sum("quantity_products").over(window_emp_total).desc())

# Aplicar funciones de ventana
df_gold = df_silver \
    .withColumn("total_by_neighborhood", F.sum("quantity_products").over(window_neigh)) \
    .withColumn("total_by_employee", F.sum("quantity_products").over(window_emp_total)) \
    .withColumn("rank_employee_quantity", F.dense_rank().over(window_rank_global)) #\
    #.withColumn("cume_dist_employee", F.cume_dist().over(window_rank_global))

In [0]:
from pyspark.sql import functions as F
from pyspark.sql.window import Window

# Ventanas para gold
# Partimos de Silver
df_silver = spark.table("poctesting.silver_events")

# Ventanas
window_neigh = Window.partitionBy("neighborhood")
window_neigh_month = Window.partitionBy("neighborhood", "event_year", "event_month")
window_rank = Window.partitionBy("event_month").orderBy(F.sum("quantity_products").over(window_neigh_month).desc())
window_day = Window.partitionBy("neighborhood", "event_year", "event_month", "event_day")

# Construcción de métricas
df_gold = df_silver \
    .withColumn("total_by_neighborhood", F.sum("quantity_products").over(window_neigh)) \
    .withColumn("avg_by_neighborhood", F.avg("quantity_products").over(window_neigh)) \
    .withColumn("count_orders_by_neigh", F.count("order_id").over(window_neigh)) \
    .withColumn("unique_customers_by_neigh", F.approx_count_distinct("customer_id").over(window_neigh)) \
    .withColumn("monthly_total", F.sum("quantity_products").over(window_neigh_month)) \
    .withColumn("monthly_avg", F.avg("quantity_products").over(window_neigh_month)) \
    .withColumn("rank_in_month", F.dense_rank().over(window_rank)) \
    .withColumn("orders_per_day", F.count("order_id").over(window_day))

# Persistir en tabla Gold
df_gold.write.mode("overwrite").saveAsTable("poctesting.gold_events")

print(f"✅ Tabla 'poctesting.gold_events' creada con {df_gold.count()} registros enriquecidos.")



In [0]:
display(df_gold.limit(10))

## Graficos
Realizamos los siguientes graficos como aporte a la solución planteada en la POC de UnalWater
1. Grafico de dispersión de los puntos
2. Mapa de cloropletas por barrio
3. Mapa de densidad de Kernel
4. Histograma de cantidad de productos vendidos
5. Histograma de productos vendidos por horas

Primero debemos leer el dataframe desde silver y recuperar la geometría de los puntos y también corregir la geometría de los barrios, que tiene huecos

In [0]:
from shapely.validation import make_valid
import geopandas as gpd

path_parquet_neigh = "/Workspace/Users/danielale22rojas@gmail.com//medellin-bigdata-poc/data/raw/medellin_neighborhoods.parquet"
gdf_barrios = gpd.read_parquet(path_parquet_neigh) 

gdf_barrios["geometry"] = gdf_barrios["geometry"].apply(make_valid)

# Primero corregimos el nombre
gdf_barrios.loc[gdf_barrios["NOMBRE"].isna(), "NOMBRE"] = "ARANJUEZ"

# Luego disolvemos por el nombre
gdf_barrios = gdf_barrios.dissolve(by="NOMBRE", as_index=False)

gdf_barrios["NOMBRE"] = gdf_barrios["NOMBRE"].str.replace("CORREGIMIENTO DE ", "", regex=True)
gdf_barrios["NOMBRE"]

In [0]:
import geopandas as gpd
from shapely.geometry import Point

def cargar_silver_como_gdf(nombre_tabla="poctesting.gold_events", crs="EPSG:4326"):
    """
    Carga la tabla Silver desde Spark y la convierte en un GeoDataFrame.
    """
    # Leer la tabla desde Spark
    df_gold = spark.table(nombre_tabla)
    
    # Pasar a Pandas
    pdf = df_gold.toPandas()

    # Crear el "geometry" a partir de lon y lat
    pdf["geometry"] = pdf.apply(lambda row: Point(row["longitude"], row["latitude"]), axis=1)
    
    # Construir GeoDataFrame
    gdf = gpd.GeoDataFrame(pdf, geometry="geometry", crs=crs)
    
    return gdf


In [0]:
# bd es la base de datos gold importada como un GeodataFrame de pandas
bd = cargar_silver_como_gdf("poctesting.gold_events", "EPSG:4326")

### 1. Grafico de dispersión de los puntos

In [0]:
import matplotlib.pyplot as plt
import seaborn as sbn

plot = sbn.jointplot(
    x='longitude', 
    y='latitude', 
    data=bd, 
    s=5,  # Tamaño de los puntos
    height=8
)

# Agregar título y etiquetas
plot.fig.suptitle("Distribución de puntos de ventas", y=1.02, fontsize=16)
plot.set_axis_labels("Longitud", "Latitud")

plt.show()


In [0]:
import matplotlib.pyplot as plt
import seaborn as sbn
import geopandas as gpd

# 2. Crear el jointplot
plot = sbn.jointplot(
    x='longitude', 
    y='latitude', 
    data=bd, 
    s=5, 
    height=8
)

# 3. Obtener el eje principal del jointplot
ax = plot.ax_joint

# 4. Dibujar los límites de barrios encima
gdf_barrios.boundary.plot(ax=ax, color="black", linewidth=0.5)

# 5. Personalizar títulos y etiquetas
plot.fig.suptitle("Distribución de puntos de venta con límites de barrios", y=1.02, fontsize=16)
plot.set_axis_labels("Longitud", "Latitud")

plt.show()


### 2. Mapa de cloropletas por barrio

In [0]:
df_gold = spark.table("poctesting.gold_events")
display(df_gold.limit(10))

In [0]:
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt

# 1. Cargar los polígonos de los barrios
gdf = gpd.read_parquet(path_parquet_neigh) 
gdf = gdf[["NOMBRE", "geometry"]]

# 2. Leer Gold y dejar un registro por barrio
pdf_gold = df_gold.select("neighborhood", "avg_by_neighborhood").distinct().toPandas()

# 3. Unir con datos espaciales
gdf_merged = gdf.merge(pdf_gold, left_on="NOMBRE", right_on="neighborhood", how="left")
gdf_merged["NOMBRE"] = gdf_merged["NOMBRE"].str.replace("CORREGIMIENTO DE ", "", regex=True)

# 4. Crear gráfico
fig, ax = plt.subplots(figsize=(16, 10))
gdf_merged.plot(
    column="avg_by_neighborhood",
    cmap="OrRd",
    edgecolor="black",
    legend=True,
    ax=ax
)

# 5. Rotular barrios usando centroide
for idx, row in gdf_merged.iterrows():
    if row["geometry"] is not None:
        centroid = row["geometry"].centroid
        plt.text(
            centroid.x,
            centroid.y,
            row["NOMBRE"],
            fontsize=6,
            ha="center",
            va="center"
        )

plt.title("Total de Productos por Barrio - Medellín", fontsize=15)
plt.axis("off")
plt.show()

# Quitar la palabra corregimiento de los nombres

In [0]:
gdf_merged.head()

In [0]:
# Unir con los totales (eliminamos duplicados porque total_by_neighborhood se repite por punto)
totales = bd[["neighborhood", "total_by_neighborhood"]].drop_duplicates()
gdf_clorop = gdf_barrios.merge(totales, left_on="NOMBRE", right_on="neighborhood")

In [0]:
import matplotlib.pyplot as plt

# 3. Graficar coroplético
f, ax = plt.subplots(1, figsize=(12,7))
gdf_clorop.plot(
    ax=ax,
    column="total_by_neighborhood",  # usamos el total ya calculado
    legend=True,
    scheme="Quantiles",
    legend_kwds={"fmt": "{:.0f}"},
    cmap="Blues",
    edgecolor="black",
    linewidth=0.5
)
ax.set_axis_off()
ax.set_title("Total de productos vendidos por barrio", fontsize=14)
plt.axis("equal")
plt.show()

### 3. Mapa de densidad de Kernel

In [0]:
import matplotlib.pyplot as plt
import seaborn as sbn
import geopandas as gpd
from matplotlib.cm import ScalarMappable

# 2. Crear figura
f, ax = plt.subplots(figsize=(10, 12))

# 3. KDE con seaborn
sns_plot = sbn.kdeplot(
    x=bd["longitude"], 
    y=bd["latitude"], 
    fill=True, 
    cmap="viridis_r", 
    levels=30, 
    alpha=0.7, 
    ax=ax
)

# 4. Dibujar límites de barrios
gdf_barrios.boundary.plot(ax=ax, color="black", linewidth=0.3, alpha=0.5)

# 5. Añadir nombres de barrios
for idx, row in gdf_barrios.iterrows():
    centroid = row.geometry.centroid
    ax.text(
        centroid.x, centroid.y, 
        str(row["NOMBRE"]), 
        fontsize=7, color="black", ha="center"
    )

# 6. Ajustar límites al bounding box de Medellín
bounds = gdf_barrios.total_bounds
ax.set_xlim(bounds[0], bounds[2])
ax.set_ylim(bounds[1], bounds[3])

# 7. Crear colorbar manual (gradiente)
sm = ScalarMappable(cmap="viridis_r")
sm.set_array([])  # necesario para inicializar
cbar = f.colorbar(sm, ax=ax, orientation="vertical", fraction=0.03, pad=0.04)
cbar.set_label("Densidad estimada de eventos", fontsize=12)

# 8. Estilo final
ax.set_title("Mapa de densidad de eventos de ventas en Medellín", fontsize=16, pad=20)
ax.set_axis_off()
plt.tight_layout()
plt.show()


In [0]:
from pointpats import QStatistic

coordinates = bd[["longitude", "latitude"]].values
qstat = QStatistic(coordinates, nx = 2, ny = 2)
qstat.plot()

### 4. Histograma de cantidad de productos vendidos

### 5. Clustering

In [0]:
# CLUSTERING CON DBSCAN
from sklearn.cluster import DBSCAN
import numpy as np

# Extraer coordenadas
coords = np.array(list(zip(bd.geometry.x, bd.geometry.y)))

# DBSCAN con distancia en coordenadas
db = DBSCAN(eps=0.01, min_samples=5).fit(coords)  
bd["cluster"] = db.labels_

# Revisar resultados
print(bd["cluster"].value_counts())

# Graficar
import matplotlib.pyplot as plt
fig, ax = plt.subplots(figsize=(10,8))
bd.plot(ax=ax, column="cluster", categorical=True, legend=True, markersize=10, cmap="tab20")
ax.set_title("Clustering de eventos (DBSCAN)", fontsize=14)
plt.show()


In [0]:
# CLUSTERING CON K-MEANS
from sklearn.cluster import KMeans

coords = np.array(list(zip(bd.geometry.x, bd.geometry.y)))
kmeans = KMeans(n_clusters=5, random_state=42).fit(coords)
bd["cluster"] = kmeans.labels_

# Graficar
fig, ax = plt.subplots(figsize=(10,8))
bd.plot(ax=ax, column="cluster", categorical=True, legend=True, markersize=10, cmap="tab20")
ax.set_title("Clustering de eventos (K-Means)", fontsize=14)
plt.show()


In [0]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10,6))
plt.hist(bd["quantity_products"], bins=30, color="skyblue", edgecolor="black")
plt.title("Histograma de productos vendidos - Poisson No homogenea", fontsize=16)
plt.xlabel("Cantidad de productos")
plt.ylabel("Frecuencia")
plt.grid(alpha=0.3)
plt.show()


In [0]:
plt.figure(figsize=(14,7))
sbn.boxplot(data=bd, x="neighborhood", y="quantity_products")
plt.title("Boxplot de productos vendidos por barrio - Uniforme", fontsize=16)
plt.xlabel("Barrio")
plt.ylabel("Cantidad de productos")
plt.xticks(rotation=90)
plt.show()
